In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import patternq.query as pqq
import patternq.dataset as pqd
import patternq.reference as pqr

/Users/benjaminkamphaus/Library/Caches/pypoetry/virtualenvs/patternq-bsnIQB4Y-py3.9/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [10]:
subjects = pqd.all_subjects('tcga-brca', db_name='tcga-brca')
subjects

,subject-HLA-A-type,subject-uid,subject-race,subject-freetext-disease,subject-HLA-B-type,db-id,subject-age,subject-sex,subject-HLA-C-type,subject-dead,subject-id,subject-ethnicity
0,[hla_a_68_01_02],"[tcga-brca, 48958e0a033b76dd2cde746ec340d7b8]",:subject.race/white,Invasive breast carcinoma,"[hla_b_40_01_02, hla_b_44_02_01_03]",17592187509997,50.0,:subject.sex/female,"[hla_c_03_04_01_01, hla_c_07_04_01]",True,TCGA-AR-A1AR,NaN
1,"[hla_a_25_01_01, hla_a_31_01_02]","[tcga-brca, cdff6ccae7b3a01e06583b6eb6935647]",:subject.race/white,Invasive breast carcinoma,"[hla_b_18_01_01_02, hla_b_51_08]",17592187510001,68.0,:subject.sex/female,"[hla_c_12_03_01_01, hla_c_16_02_01]",True,TCGA-BH-A1EO,NaN
2,[hla_a_03_01_01_01],"[tcga-brca, 58de8decc5806306e122d7d2e0361077]",:subject.race/white,Invasive breast carcinoma,[hla_b_07_02_01],17592187510005,35.0,:subject.sex/female,[hla_c_07_02_01_03],True,TCGA-BH-A1ES,NaN
3,[hla_a_26_01_03],"[tcga-brca, e33c10ac862066f01799593f93226d63]",:subject.race/white,Invasive breast carcinoma,"[hla_b_38_01_01, hla_b_40_02_01]",17592187510009,55.0,:subject.sex/female,"[hla_c_02_02_02, hla_c_12_03_01_01]",True,TCGA-BH-A1ET,NaN
4,[hla_a_02_01_01_01],"[tcga-brca, 33c8d0592a63e6d929493c6fee83049a]",:subject.race/white,Invasive breast carcinoma,"[hla_b_13_02_01, hla_b_18_01_01_02]",17592187510013,83.0,:subject.sex/female,"[hla_c_06_02_01_01, hla_c_07_01_01_01]",True,TCGA-BH-A1EU,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1093,"[hla_a_01_01_01_01, hla_a_26_01_01]","[tcga-brca, 28355fea1611cf76788e36f7d8d1e63b]",:subject.race/white,Invasive breast carcinoma,"[hla_b_07_02_01, hla_b_08_01_07]",17592187514126,44.0,:subject.sex/female,"[hla_c_07_01_01_01, hla_c_07_02_01_03]",False,TCGA-E2-A1B6,NaN
1094,"[hla_a_01_01_01_01, hla_a_24_02_01_01]","[tcga-brca, b49d76c1ed3ce2f807c30aca593700e5]",NaN,Invasive breast carcinoma,"[hla_b_08_01_01, hla_b_44_05_01]",17592187514130,63.0,:subject.sex/female,"[hla_c_02_02_02, hla_c_07_01_01_01]",False,TCGA-E2-A1BC,NaN
1095,[hla_a_02_01_01_01],"[tcga-brca, 679b389e478544f624e010f2fba4951c]",:subject.race/white,Invasive breast carcinoma,"[hla_b_07_02_01, hla_b_35_01_01_02]",17592187514133,53.0,:subject.sex/female,"[hla_c_04_01_01_01, hla_c_07_02_01_03]",False,TCGA-E2-A1BD,NaN
1096,"[hla_a_31_01_02, hla_a_80_01_01_01]","[tcga-brca, ac707e57a213016c5f7f1a643c938bc0]",:subject.race/african-american,Invasive breast carcinoma,"[hla_b_27_05_02, hla_b_40_01_02]",17592187517522,NaN,:subject.sex/female,"[hla_c_03_04_01_01, hla_c_06_02_01_01]",False,TCGA-OL-A66H,:subject.ethnicity/not-hispanic


In [18]:
by_hla = subjects[['subject-id', 'subject-HLA-A-type', 'subject-HLA-B-type', 'subject-HLA-C-type']].explode(column='subject-HLA-A-type')
by_hla = by_hla.explode(column='subject-HLA-B-type')
by_hla = by_hla.explode(column='subject-HLA-C-type')
by_hla

,subject-id,subject-HLA-A-type,subject-HLA-B-type,subject-HLA-C-type
0,TCGA-AR-A1AR,hla_a_68_01_02,hla_b_40_01_02,hla_c_03_04_01_01
0,TCGA-AR-A1AR,hla_a_68_01_02,hla_b_40_01_02,hla_c_07_04_01
0,TCGA-AR-A1AR,hla_a_68_01_02,hla_b_44_02_01_03,hla_c_03_04_01_01
0,TCGA-AR-A1AR,hla_a_68_01_02,hla_b_44_02_01_03,hla_c_07_04_01
1,TCGA-BH-A1EO,hla_a_25_01_01,hla_b_18_01_01_02,hla_c_12_03_01_01
...,...,...,...,...
1096,TCGA-OL-A66H,hla_a_80_01_01_01,hla_b_27_05_02,hla_c_03_04_01_01
1096,TCGA-OL-A66H,hla_a_80_01_01_01,hla_b_27_05_02,hla_c_06_02_01_01
1096,TCGA-OL-A66H,hla_a_80_01_01_01,hla_b_40_01_02,hla_c_03_04_01_01
1096,TCGA-OL-A66H,hla_a_80_01_01_01,hla_b_40_01_02,hla_c_06_02_01_01


In [27]:
by_hla['subject-HLA-B-type'].value_counts()

subject-HLA-B-type
hla_b_07_02_01       841
hla_b_08_01_01       508
hla_b_15_01_01_01    423
hla_b_44_02_01_01    393
hla_b_35_01_01_02    332
                    ... 
hla_b_51_07_02         2
hla_b_39_01_03         1
hla_b_44_02_15         1
hla_b_14_03            1
hla_b_07_02_06         1
Name: count, Length: 102, dtype: int64

In [33]:
b27_matches = by_hla['subject-HLA-B-type'].str.contains("b_27")
b27_matches = b27_matches.fillna(False)
b27_matches = b27_matches == True
b27_sub = by_hla[b27_matches]
b27_sub

/var/folders/ym/gv3tr8lj5g3773_33yn9gr200000gn/T/ipykernel_60899/4069950816.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  b27_matches = b27_matches.fillna(False)


,subject-id,subject-HLA-A-type,subject-HLA-B-type,subject-HLA-C-type
15,TCGA-E2-A15I,hla_a_03_01_19,hla_b_27_05_02,hla_c_01_02_01
15,TCGA-E2-A15I,hla_a_03_01_19,hla_b_27_05_02,hla_c_15_02_01
15,TCGA-E2-A15I,hla_a_26_01_03,hla_b_27_05_02,hla_c_01_02_01
15,TCGA-E2-A15I,hla_a_26_01_03,hla_b_27_05_02,hla_c_15_02_01
33,TCGA-B6-A1KN,hla_a_03_01_01_01,hla_b_27_05_02,hla_c_02_02_02
...,...,...,...,...
1064,TCGA-AR-A1AQ,hla_a_02_06_05,hla_b_27_02_01,hla_c_16_01_01
1096,TCGA-OL-A66H,hla_a_31_01_02,hla_b_27_05_02,hla_c_03_04_01_01
1096,TCGA-OL-A66H,hla_a_31_01_02,hla_b_27_05_02,hla_c_06_02_01_01
1096,TCGA-OL-A66H,hla_a_80_01_01_01,hla_b_27_05_02,hla_c_03_04_01_01


In [35]:
sub_id = "TCGA-B6-A1KN"
assays_for_sub = pqd.assays_for_patient(sub_id, db_name='tcga-brca')
assays_for_sub

,subject-id,sample-id,assay-tech,assay-name,measurement-set-name
0,TCGA-B6-A1KN,tcga-b6-a1kn-01a,:assay.technology/RNA-seq,RNA-seq data,gx
1,TCGA-B6-A1KN,tcga-b6-a1kn-01a,:assay.technology/SNP-array,cnv and variants,snp CNV data
2,TCGA-B6-A1KN,tcga-b6-a1kn-01a,:assay.technology/WES,WES,baseline mutations
3,TCGA-B6-A1KN,tcga-b6-a1kn-10a,:assay.technology/RNA-seq,RNA-seq data,immune-cell-deconvolution
4,TCGA-B6-A1KN,tcga-b6-a1kn-01a,:assay.technology/RNA-seq,RNA-seq data,immune-cell-deconvolution
5,TCGA-B6-A1KN,tcga-b6-a1kn-01a,:assay.technology/miscellaneous,Tumor purity,tumor purity
6,TCGA-B6-A1KN,tcga-b6-a1kn-10a,:assay.technology/SNP-array,cnv and variants,snp CNV data


In [36]:
samples = pqd.samples("tcga-brca", db_name='tcga-brca')
samples

,sample-id,sample-tumor-type,sample-specimen,sample-metastasis,sample-uid,db-id,sample-timepoint-timepoint-id,sample-subject-subject-id,sample-study-day-study-day-id,sample-type-db-id,sample-gdc-anatomic-site-gdc-anatomic-site-name
0,tcga-a2-a0eu-01a,breast,:sample.specimen/fresh-frozen,False,"[tcga-brca, 1083184581b3c24ac85291d75a832b9a]",17592187518720,sample procurement,TCGA-A2-A0EU,35,17592186045420,Breast
1,tcga-a2-a0eu-10a,NaN,:sample.specimen/fresh-frozen,False,"[tcga-brca, e96e9d5e9e41bba015ba8e4daefda967]",17592187518803,sample procurement,TCGA-A2-A0EU,35,17592186045421,Breast
2,tcga-a7-a0d9-01a,breast,:sample.specimen/fresh-frozen,False,"[tcga-brca, 8c1dbb6ba31a61b3b978a1f93dccb86c]",17592187518823,sample procurement,TCGA-A7-A0D9,10,17592186045420,Breast
3,tcga-a7-a0d9-10a,NaN,:sample.specimen/fresh-frozen,False,"[tcga-brca, ba8add19b462211346377f6c7835cb6a]",17592187519258,sample procurement,TCGA-A7-A0D9,10,17592186045421,Breast
4,tcga-ao-a0jf-01a,breast,:sample.specimen/fresh-frozen,False,"[tcga-brca, 0bffbcada51f70f1978d207e22089d60]",17592187519626,sample procurement,TCGA-AO-A0JF,57,17592186045420,Breast
...,...,...,...,...,...,...,...,...,...,...,...
2291,tcga-b6-a402-10a,NaN,:sample.specimen/fresh-frozen,False,"[tcga-brca, 9ede5e348369b0d13d4c7989eb565e3c]",17592188110342,sample procurement,TCGA-B6-A402,0,17592186045421,Breast
2292,tcga-e9-a1ne-11a,NaN,:sample.specimen/fresh-frozen,False,"[tcga-brca, 85358f998e91a477db9bc868554ea46c]",17592188110466,sample procurement,TCGA-E9-A1NE,0,17592186045421,Breast
2293,tcga-ol-a5da-10a,NaN,:sample.specimen/fresh-frozen,False,"[tcga-brca, 7882611517385fe91546199a73b523c7]",17592188111439,sample procurement,TCGA-OL-A5DA,33,17592186045421,Breast
2294,tcga-bh-a5j0-10a,NaN,:sample.specimen/fresh-frozen,False,"[tcga-brca, 934e529d67c8a4b6e7840615131ce315]",17592188112629,sample procurement,TCGA-BH-A5J0,104,17592186045421,Breast


In [37]:
samples_sub = samples[samples["sample-subject-subject-id"] == sub_id]
samples_sub

,sample-id,sample-tumor-type,sample-specimen,sample-metastasis,sample-uid,db-id,sample-timepoint-timepoint-id,sample-subject-subject-id,sample-study-day-study-day-id,sample-type-db-id,sample-gdc-anatomic-site-gdc-anatomic-site-name
105,tcga-b6-a1kn-01a,breast,:sample.specimen/fresh-frozen,False,"[tcga-brca, 83fe2368405c6e3d2a08288d83f8f0db]",17592187547478,sample procurement,TCGA-B6-A1KN,36,17592186045420,Breast
106,tcga-b6-a1kn-10a,NaN,:sample.specimen/fresh-frozen,False,"[tcga-brca, 870c4800388cc94b29d58203db3a065b]",17592187548097,sample procurement,TCGA-B6-A1KN,36,17592186045421,Breast


In [38]:
clin_sum = pqd.clinical_summary("tcga-brca", db_name='tcga-brca')
clin_sum

,clinical-observation-set-name
0,pfs os recurrence


In [39]:
sample_for_meas = assays_for_sub[assays_for_sub['measurement-set-name'] == 'baseline mutations']
sample_for_meas = sample_for_meas.iloc[0]['sample-id']
sample_meas = pqd.sample_measurements('tcga-brca', 'baseline mutations', sample_for_meas, db_name='tcga-brca')
sample_meas


,measurement-variant-variant-id,measurement-sample-sample-id
0,GRCh37:chr3:+:10500146:10500146/C/T,tcga-b6-a1kn-01a
1,GRCh37:chr7:+:25228619:25228619/G/T,tcga-b6-a1kn-01a
2,GRCh37:chr20:+:62917053:62917053/G/A,tcga-b6-a1kn-01a
3,GRCh37:chr11:+:41032794:41032794/C/T,tcga-b6-a1kn-01a
4,GRCh37:chr4:+:24989509:24989509/G/A,tcga-b6-a1kn-01a
5,GRCh37:chr20:+:129155089:129155089/C/A,tcga-b6-a1kn-01a
6,GRCh37:chr13:+:9783057:9783057/T/G,tcga-b6-a1kn-01a
7,GRCh37:chr3:+:68030078:68030078/C/T,tcga-b6-a1kn-01a
8,GRCh37:chr8:+:30665245:30665245/G/A,tcga-b6-a1kn-01a
9,GRCh37:chr19:+:43853546:43853546/C/T,tcga-b6-a1kn-01a


In [40]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:

var = pqr.variants(sample_meas['measurement-variant-variant-id'].tolist(), db_name='tcga-brca')
var

,variant-ref-allele,variant-id,variant-alt-allele,db-id,variant-classification,variant-type-db-id,variant-gene-gene-hgnc-symbol,variant-genomic-coordinates-db-id,unify-import-most-recent-db-id
0,C,GRCh37:chr3:+:10500146:10500146/C/T,T,17592187439906,:variant.classification/missense,17592186045584,OR5D14,17592186746250,17592186177812
1,G,GRCh37:chr7:+:25228619:25228619/G/T,T,17592187439908,:variant.classification/missense,17592186045584,ZCCHC12,17592186746252,17592186177812
2,G,GRCh37:chr20:+:62917053:62917053/G/A,A,17592187439910,:variant.classification/silent,17592186045584,FMO3,17592186746254,17592186177812
3,C,GRCh37:chr11:+:41032794:41032794/C/T,T,17592187439912,:variant.classification/missense,17592186045584,ATP13A2,17592186746256,17592186177812
4,G,GRCh37:chr4:+:24989509:24989509/G/A,A,17592187439914,:variant.classification/missense,17592186045584,CPN2,17592186746259,17592186177812
5,C,GRCh37:chr20:+:129155089:129155089/C/A,A,17592187439917,:variant.classification/missense,17592186045584,CNTNAP5,17592186746261,17592186177812
6,T,GRCh37:chr13:+:9783057:9783057/T/G,G,17592187439919,:variant.classification/missense,17592186045584,MST1R,17592186746263,17592186177812
7,C,GRCh37:chr3:+:68030078:68030078/C/T,T,17592187439921,:variant.classification/missense,17592186045584,CERS1,17592186746265,17592186177812
8,G,GRCh37:chr8:+:30665245:30665245/G/A,A,17592187439923,:variant.classification/missense,17592186045584,APOBEC3B,17592186746267,17592186177812
9,C,GRCh37:chr19:+:43853546:43853546/C/T,T,17592187439925,:variant.classification/missense,17592186045584,GPAT2,17592186746269,17592186177812


In [43]:
w_var = sample_meas.merge(
    var[['variant-id',
         'variant-ref-allele',
         'variant-alt-allele',
         'variant-gene-gene-hgnc-symbol',
         'variant-classification']],
    left_on='measurement-variant-variant-id',
    right_on='variant-id',
    how='left'
)
w_var

,measurement-variant-variant-id,measurement-sample-sample-id,variant-id,variant-ref-allele,variant-alt-allele,variant-gene-gene-hgnc-symbol,variant-classification
0,GRCh37:chr3:+:10500146:10500146/C/T,tcga-b6-a1kn-01a,GRCh37:chr3:+:10500146:10500146/C/T,C,T,OR5D14,:variant.classification/missense
1,GRCh37:chr7:+:25228619:25228619/G/T,tcga-b6-a1kn-01a,GRCh37:chr7:+:25228619:25228619/G/T,G,T,ZCCHC12,:variant.classification/missense
2,GRCh37:chr20:+:62917053:62917053/G/A,tcga-b6-a1kn-01a,GRCh37:chr20:+:62917053:62917053/G/A,G,A,FMO3,:variant.classification/silent
3,GRCh37:chr11:+:41032794:41032794/C/T,tcga-b6-a1kn-01a,GRCh37:chr11:+:41032794:41032794/C/T,C,T,ATP13A2,:variant.classification/missense
4,GRCh37:chr4:+:24989509:24989509/G/A,tcga-b6-a1kn-01a,GRCh37:chr4:+:24989509:24989509/G/A,G,A,CPN2,:variant.classification/missense
5,GRCh37:chr20:+:129155089:129155089/C/A,tcga-b6-a1kn-01a,GRCh37:chr20:+:129155089:129155089/C/A,C,A,CNTNAP5,:variant.classification/missense
6,GRCh37:chr13:+:9783057:9783057/T/G,tcga-b6-a1kn-01a,GRCh37:chr13:+:9783057:9783057/T/G,T,G,MST1R,:variant.classification/missense
7,GRCh37:chr3:+:68030078:68030078/C/T,tcga-b6-a1kn-01a,GRCh37:chr3:+:68030078:68030078/C/T,C,T,CERS1,:variant.classification/missense
8,GRCh37:chr8:+:30665245:30665245/G/A,tcga-b6-a1kn-01a,GRCh37:chr8:+:30665245:30665245/G/A,G,A,APOBEC3B,:variant.classification/missense
9,GRCh37:chr19:+:43853546:43853546/C/T,tcga-b6-a1kn-01a,GRCh37:chr19:+:43853546:43853546/C/T,C,T,GPAT2,:variant.classification/missense


In [45]:
w_var[w_var['variant-classification'] == ':variant.classification/missense']

,measurement-variant-variant-id,measurement-sample-sample-id,variant-id,variant-ref-allele,variant-alt-allele,variant-gene-gene-hgnc-symbol,variant-classification
0,GRCh37:chr3:+:10500146:10500146/C/T,tcga-b6-a1kn-01a,GRCh37:chr3:+:10500146:10500146/C/T,C,T,OR5D14,:variant.classification/missense
1,GRCh37:chr7:+:25228619:25228619/G/T,tcga-b6-a1kn-01a,GRCh37:chr7:+:25228619:25228619/G/T,G,T,ZCCHC12,:variant.classification/missense
3,GRCh37:chr11:+:41032794:41032794/C/T,tcga-b6-a1kn-01a,GRCh37:chr11:+:41032794:41032794/C/T,C,T,ATP13A2,:variant.classification/missense
4,GRCh37:chr4:+:24989509:24989509/G/A,tcga-b6-a1kn-01a,GRCh37:chr4:+:24989509:24989509/G/A,G,A,CPN2,:variant.classification/missense
5,GRCh37:chr20:+:129155089:129155089/C/A,tcga-b6-a1kn-01a,GRCh37:chr20:+:129155089:129155089/C/A,C,A,CNTNAP5,:variant.classification/missense
6,GRCh37:chr13:+:9783057:9783057/T/G,tcga-b6-a1kn-01a,GRCh37:chr13:+:9783057:9783057/T/G,T,G,MST1R,:variant.classification/missense
7,GRCh37:chr3:+:68030078:68030078/C/T,tcga-b6-a1kn-01a,GRCh37:chr3:+:68030078:68030078/C/T,C,T,CERS1,:variant.classification/missense
8,GRCh37:chr8:+:30665245:30665245/G/A,tcga-b6-a1kn-01a,GRCh37:chr8:+:30665245:30665245/G/A,G,A,APOBEC3B,:variant.classification/missense
9,GRCh37:chr19:+:43853546:43853546/C/T,tcga-b6-a1kn-01a,GRCh37:chr19:+:43853546:43853546/C/T,C,T,GPAT2,:variant.classification/missense
11,GRCh37:chr14:+:187130348:187130348/C/T,tcga-b6-a1kn-01a,GRCh37:chr14:+:187130348:187130348/C/T,C,T,KRTAP10-10,:variant.classification/missense
